# Context Injection - Dynamic Data Flow

This notebook demonstrates how to inject custom context and data into the sandbox execution.

**Use cases:**
- Pass API responses to be processed
- Inject database query results
- Provide user data for transformations
- Supply configuration objects

**Documentation:** [JSR Package](https://jsr.io/@casys/mcp-gateway)

## Setup Sandbox

In [ ]:
import { DenoSandboxExecutor } from "jsr:@casys/mcp-gateway";

const sandbox = new DenoSandboxExecutor({
  timeout: 5000,
  memoryLimit: 128,
});

console.log("✅ Sandbox ready");

## 1. Basic Context Injection

Inject simple data that the code can access via the global `context` variable:

In [ ]:
const code = `
// Context is available as a global variable
return {
  receivedName: context.name,
  receivedAge: context.age,
  message: \`Hello \${context.name}, you are \${context.age} years old\`
};
`;

const result = await sandbox.execute(code, {
  name: "Alice",
  age: 30
});

console.log(JSON.stringify(result, null, 2));

## 2. Processing API-like Data

Simulate processing an API response with complex nested data:

In [ ]:
const apiResponse = {
  users: [
    { id: 1, name: "Alice", email: "alice@example.com", active: true },
    { id: 2, name: "Bob", email: "bob@example.com", active: false },
    { id: 3, name: "Charlie", email: "charlie@example.com", active: true },
  ],
  metadata: {
    total: 3,
    timestamp: "2025-01-15T10:30:00Z"
  }
};

const code = `
// Filter active users and extract emails
const activeUsers = context.users.filter(u => u.active);

return {
  activeCount: activeUsers.length,
  activeEmails: activeUsers.map(u => u.email),
  summary: \`\${activeUsers.length} out of \${context.metadata.total} users are active\`,
  processedAt: new Date().toISOString()
};
`;

const result = await sandbox.execute(code, apiResponse);
console.log(JSON.stringify(result, null, 2));

## 3. Data Transformation Pipeline

Transform data through multiple operations:

In [ ]:
const salesData = {
  transactions: [
    { product: "Laptop", price: 1200, quantity: 2, date: "2025-01-10" },
    { product: "Mouse", price: 25, quantity: 5, date: "2025-01-11" },
    { product: "Keyboard", price: 80, quantity: 3, date: "2025-01-12" },
    { product: "Monitor", price: 300, quantity: 1, date: "2025-01-13" },
  ]
};

const code = `
// Calculate totals and aggregate statistics
const transactions = context.transactions;

const totals = transactions.map(t => ({
  ...t,
  total: t.price * t.quantity
}));

const grandTotal = totals.reduce((sum, t) => sum + t.total, 0);
const avgTransaction = grandTotal / totals.length;

// Find best seller
const bestSeller = totals.reduce((best, current) => 
  current.total > best.total ? current : best
);

return {
  totals,
  summary: {
    grandTotal,
    avgTransaction: Math.round(avgTransaction * 100) / 100,
    transactionCount: totals.length,
    bestSeller: {
      product: bestSeller.product,
      revenue: bestSeller.total
    }
  }
};
`;

const result = await sandbox.execute(code, salesData);
console.log(JSON.stringify(result, null, 2));

## 4. Configuration-Driven Execution

Use context to control code behavior dynamically:

In [ ]:
const config = {
  data: [10, 25, 30, 15, 40, 5],
  operation: "filter",  // Try: 'filter', 'sort', 'transform'
  threshold: 20,
  sortOrder: "desc"
};

const code = `
let result = context.data;

switch (context.operation) {
  case 'filter':
    result = result.filter(x => x > context.threshold);
    break;
  
  case 'sort':
    result = [...result].sort((a, b) => 
      context.sortOrder === 'desc' ? b - a : a - b
    );
    break;
  
  case 'transform':
    result = result.map(x => x * 2);
    break;
}

return {
  operation: context.operation,
  original: context.data,
  result,
  count: result.length
};
`;

const result = await sandbox.execute(code, config);
console.log(JSON.stringify(result, null, 2));

## 5. Error Handling with Context

Gracefully handle missing or invalid context:

In [ ]:
const code = `
// Validate context before using
if (!context || !context.data) {
  throw new Error("Missing required context.data");
}

if (!Array.isArray(context.data)) {
  throw new Error("context.data must be an array");
}

return {
  sum: context.data.reduce((a, b) => a + b, 0),
  count: context.data.length
};
`;

// Test with invalid context
const result1 = await sandbox.execute(code, { data: "not an array" });
console.log("Invalid context:");
console.log(JSON.stringify(result1, null, 2));

// Test with valid context
const result2 = await sandbox.execute(code, { data: [1, 2, 3, 4, 5] });
console.log("\nValid context:");
console.log(JSON.stringify(result2, null, 2));

## Summary

Context injection enables:
- ✅ **Dynamic data processing** - Transform any JSON data
- ✅ **Configuration-driven code** - Control behavior via context
- ✅ **API integration** - Process responses safely
- ✅ **Validation** - Check context before use
- ✅ **Type safety** - Structure your context with TypeScript interfaces

**Next:** See `security-demo.ipynb` for resource limits and permissions